In [ ]:
%reset

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime as dt
from matplotlib import pyplot as plt
from scipy import signal
from scipy.signal import butter, lfilter, freqz,filtfilt, welch,find_peaks
from scipy.fft import fft 
from scipy.special import entr
from scipy.stats import kurtosis,skew,entropy
from scipy.stats.stats import pearsonr 
from scipy.fftpack import fft
import math
import time
import seaborn as sns
import pickle
from statistics import stdev
from statistics import variance
import random
from detecta import detect_peaks
import pywt 

In [ ]:
data = pd.read_csv('all_user.csv')
data.label.value_counts()
print(data.shape)

In [ ]:
data.datetime = pd.to_datetime(data.datetime)
data.set_index('datetime', inplace =True)
data = data.tz_convert(tz='UTC')
data = data.sort_values(by='datetime')
data.reset_index(inplace=True)
data.head()

In [ ]:
plt.plot(data.loc[0:400,'x'])

In [ ]:
#mag1 = (data['x']**2 + data['y']**2 + data['z']**2)**(1/2)
#data['magnitude'] = mag1
#print(data.head())
#print(data.shape)

In [ ]:
#def butter_lowpass(cutoff, samp_freq, order = 3):
 #   nyq_freq = 0.5 * samp_freq
  #  normal_cutoff = cutoff/nyq_freq
   # b,a = butter(order, normal_cutoff, btype = 'lowpass', analog = False)
   # return b, a

#def butter_lowpass_filter(data, cutoff, samp_freq, order = 3):
 #   b, a = butter_lowpass(cutoff, samp_freq, order)
 #   y = filtfilt(b, a, data)
 #   return y

#order = 3
#cutoff = 1
#samp_freq = 20
#b, a = butter_lowpass(cutoff, samp_freq, order)

#gra_x1 = butter_lowpass_filter(data.x, cutoff, samp_freq, order=3)
#gra_y1 = butter_lowpass_filter(data.y, cutoff, samp_freq, order=3)
#gra_z1 = butter_lowpass_filter(data.z, cutoff, samp_freq, order=3)
#gra_mag1 = butter_lowpass_filter(data.magnitude, cutoff, samp_freq, order=3)



#data['gra_x'] = gra_x1
#data['gra_y'] = gra_y1
#data['gra_z'] = gra_z1
#data['gra_mag'] = gra_mag1




In [ ]:
#def butter_highpass(cutoff, samp_freq, order = 3):
 #   nyq_freq = 0.5 * samp_freq
 #   normal_cutoff = cutoff/nyq_freq
 #   b,a = butter(order, normal_cutoff, btype = 'highpass', analog = False)
 #   return b, a

#def butter_highpass_filter(data, cutoff, samp_freq, order = 3):
#    b, a = butter_highpass(cutoff, samp_freq, order)
#    y = filtfilt(b, a, data)
#    return y

#order = 3
#cutoff = 1
#samp_freq = 20
#b, a = butter_highpass(cutoff, samp_freq, order)

#body_x1 = butter_highpass_filter(data.x, cutoff, samp_freq, order=3)
#body_y1 = butter_highpass_filter(data.y, cutoff, samp_freq, order=3)
#body_z1 = butter_highpass_filter(data.z, cutoff, samp_freq, order=3)
#body_mag1 = butter_highpass_filter(data.magnitude, cutoff, samp_freq, order=3)



#data['body_x'] = body_x1
#data['body_y'] = body_y1
#data['body_z'] = body_z1
#data['body_mag'] = body_mag1


In [ ]:

#time = 1/samp_freq
    
#data['jerk_x'] = (data.x.diff(periods = 1))/time
#data['jerk_y'] = (data.y.diff(periods = 1))/time
#data['jerk_z'] = (data.z.diff(periods = 1))/time
#data['jerk_magnitude'] = (data.magnitude.diff(periods = 1))/time
        
#data['jerk_gra_x'] = (data.gra_x.diff(periods = 1))/time
#data['jerk_gra_y'] = (data.gra_y.diff(periods = 1))/time
#data['jerk_gra_z'] = (data.gra_z.diff(periods = 1))/time
#data['jerk_gra_mag'] = (data.gra_mag.diff(periods = 1))/time
        
#data['jerk_body_x'] = (data.body_x.diff(periods = 1))/time
#data['jerk_body_y'] = (data.body_y.diff(periods = 1))/time
#data['jerk_body_z'] = (data.body_z.diff(periods = 1))/time
#data['jerk_body_mag'] = (data.body_mag.diff(periods = 1))/time


# Segmenting the total labelled data by different label

In [ ]:

## Segmenting the total labelled data by  different label


df = {}
for i in tqdm(range(28)):
    df[i+1] = data[(data['label'] == i+1)]
    df[i+1].datetime = pd.to_datetime(df[i+1].datetime)
    print('\n\n Label :',i+1)
    print('\n\n Head:\n\n', df[i+1].head())
    print('\n\n Tail:\n\n', df[i+1].tail())
    print('\n\n Shape:\n\n', df[i+1].shape)

# Segmenting each 'label-wise segmented' DF where frequency is not 20Hz

In [ ]:
## Segmenting different label data where frequency is not 20Hz


segment={}
#d = df[1].datetime.diff(periods=1).nunique()
#d
for i in tqdm(range(len(df))):
            df[i+1]=df[i+1].sort_values(by='datetime').reset_index(drop=True)
            com1 = dt.timedelta(milliseconds=50)
            df[i+1].datetime = pd.to_datetime(df[i+1].datetime)
            segment[i+1] = df[i+1][(df[i+1].datetime.diff(periods=1)>com1)]
            segment[i+1]=segment[i+1].sort_values(by='datetime')
            print('\n\n Segmentated dataframe of label', i+1)
            print(' That is segmentation of df ' , i+1)
            print('\n\n Head : \n',segment[i+1].head())
            print('\n\n Tail : \n',segment[i+1].tail())
            print('\n\n Shape : ', segment[i+1].shape)


# Checking if there is any label-wise segmented DF where there is no timestamp diff greater than 0.05sec


In [ ]:
exist={}
c = 1
for i in tqdm(range(len(df))):
    if  (df[i+1].datetime.diff(periods=1).nunique()==1):
            exist[c] = df[i+1]
            print('\n\n This dataframe does not require segmentation')
            print(' Number of label that does not need segmentation :' , i+1)
            print('\n\n Head : \n',exist[c].head())
            print('\n\n Tail : \n',exist[c].tail())
            print('\n\n Shape : ', exist[c].shape)
            c+=1
    else :
            continue

# Combining all the segmented DF where we can apply windowing 

In [ ]:
wi ={}
new1=0
count1=1
count2=1
for i in tqdm(range(len(segment))):
    print('\n\nThis is a part of segment no : ' , i+1)
    for j in segment[i+1].index:
        wi[count2] = pd.DataFrame()
        wi[count2]=df[i+1].loc[new1:j-1]
        print('No {0} index of d{1} is {2}'.format(count1,i+1,j))
        print('wi',count2,'\n')
        print('\n\n Head:\n',wi[count2].head())
        print('\n\n Tail:\n',wi[count2].tail())
        print('\n\n Shape:',wi[count2].shape,'\n')
        count2 +=1
        count1+=1
        new1=j
        if (j==segment[i+1].index[-1]):
            wi[count2] = df[i+1].loc[new1:]
            print('No {0} index of d{1} is {2}'.format(count1-1,i+1,j))
            print('wi',count2,'\n')
            print('\n\n Head:\n',wi[count2].head())
            print('\n\n Tail:\n',wi[count2].tail())
            print('\n\n Shape:',wi[count2].shape,'\n') 
            count1=1
            count2+=1
            new1=0
        else :
            continue
        #print('wi',count1,'\n')
        #print('\n\n Head:\n',wi[count1].head())
        #print('\n\n Tail:\n',wi[count1].tail())
        #print('\n\n Shape:',wi[count1].shape,'\n')

In [ ]:
a = len(wi)+1
print('Length of wi is : ' , len(wi))
print('Length of unsegmented dataframe is : ', len(exist))
for i in tqdm(range(len(exist))):
    wi[a]=exist[i+1]
    a+=1
print('After adding unsegmented dataframe, length of wi is : ', len(wi))

# Windowing each of the previously segmented DF

In [ ]:
pwd1 = 200
pwd2 = 50
overlap1 = 80
overlap2 = 20
final_window = pd.DataFrame(columns = ['user_id','acc_x','acc_y','acc_z','start','finish','label'])
final_window['acc_x'] = final_window['acc_x'].astype('object')
final_window['acc_y'] = final_window['acc_y'].astype('object')
final_window['acc_z'] = final_window['acc_z'].astype('object')
no1 = 0
no2 = pwd1
no3 = 0
no4 = pwd2
x_list = []
y_list = []
z_list = []
hal = 0
value_count = 0
data_count = 0
for j in tqdm(range(len(wi))):
    no1 = 0
    no2 = pwd1
    no3 = 0
    no4 = pwd2
    
    #val = math.ceil(((wi[j+1].index[-1]+1) - (wi[j+1].index[0]))/pwd)
    
    print('Length of wi{0} is {1}'.format(j+1, math.ceil(((wi[j+1].index[-1]+1) - (wi[j+1].index[0])))))
    wi[j+1] = wi[j+1].reset_index(drop=True)
    
    if (len(wi[j+1]) >= pwd1):
        print('\n\nThis wi{0} has more than {1} data \n\n'.format(j+1, pwd1))
        while(no1 <= (len(wi[j+1]) - pwd1 + overlap1)):
            if no2 <= len(wi[j+1]):
                print('\n\nNumber till which list is windowed : ',no2)
                final_window.loc[hal,'start'] = wi[j+1].loc[no1,'datetime']
                final_window.loc[hal,'finish'] = wi[j+1].loc[no2-1,'datetime']
                x_list.append(wi[j+1].loc[no1:no2-1].x)
                x_list[hal] = x_list[hal].to_numpy()
                final_window.loc[hal,'acc_x'] =  x_list[hal]
                y_list.append(wi[j+1].loc[no1:no2-1].y)
                y_list[hal] = y_list[hal].to_numpy()
                final_window.loc[hal,'acc_y'] =  y_list[hal]
                z_list.append(wi[j+1].loc[no1:no2-1].z)
                z_list[hal] = z_list[hal].to_numpy()
                final_window.loc[hal,'acc_z'] =  z_list[hal]
                final_window.loc[hal,'label'] = wi[j+1].loc[i].label
                final_window.loc[hal,'user_id'] = wi[j+1].loc[i].User_id
                no1 = no1 + overlap1
                no2 = no2 + overlap1
                print('\n\nThe \'hal\' value is : ', hal )
                hal+=1
            
            else :
                print('\n\nHere not enough data for making another window.')
                value_count += 1
                no1 = no1 + overlap1
                
                
    #elif (len(wi[j+1]) < pwd1) & (len(wi[j+1]) >= pwd2) :
     #   print('\n\nThis wi{0} has less than {1} data \n\n'.format(j+1, pwd1))
      #  while(no3 <= (len(wi[j+1]) - pwd2 + overlap2)):
      #      if no4 <= len(wi[j+1]):
      #          print('\n\nNumber till which list is windowed : ',no4)
      #          final_window.loc[hal,'start'] = wi[j+1].loc[no3,'datetime']
       #         final_window.loc[hal,'finish'] = wi[j+1].loc[no4-1,'datetime']
       #         x_list.append(wi[j+1].loc[no3:no4-1].x)
       #         x_list[hal] = x_list[hal].to_numpy()
       #         final_window.loc[hal,'acc_x'] =  x_list[hal]
       #         y_list.append(wi[j+1].loc[no3:no4-1].y)
       #         y_list[hal] = y_list[hal].to_numpy()
       #         final_window.loc[hal,'acc_y'] =  y_list[hal]
       #         z_list.append(wi[j+1].loc[no3:no4-1].z)
       #         z_list[hal] = z_list[hal].to_numpy()
       #         final_window.loc[hal,'acc_z'] =  z_list[hal]
       #         final_window.loc[hal,'label'] = wi[j+1].loc[i].label
       #         final_window.loc[hal,'user_id'] = wi[j+1].loc[i].User_id
       #         no3 = no3 + overlap2
       #         no4 = no4 + overlap2
       #         print('\n\nThe \'hal\' value is : ', hal )
       #         hal+=1
            
        #    else :
        #        print('\n\nHere not enough data for making another window.')
        #        value_count += 1
        #        no3 = no3 + overlap2
                
    else :
        data_count += 1
        
print('\n\nNo of Dataframe which has data less than desired value (50 & 100 ) : ', data_count)        
print('\n\nNo of incident when there were not sufficient data for making new widow : ', value_count )                

In [ ]:
print(final_window.shape)
final_window.head()

In [ ]:
final_window.label.value_counts()

In [ ]:
len(final_window.acc_x[0])

In [ ]:
pickel_out = open('window_pi_200.pickle','wb')
pickle.dump(final_window,pickel_out)
pickel_out.close()